# Single Hidden Layer [16] - Final Simplification Check

The simplification arc:
- Too complex: [256,128,64] CV 0.0105
- Better: [64,32] CV 0.0097
- Best so far: [32,16] CV 0.0093
- Too simple: Ridge (linear) CV 0.0115

This experiment tests [16] - the only unexplored point between [32,16] and Ridge.

Expected outcome: Likely worse than [32,16] (between 0.0093 and 0.0115)
Decision rule:
- If CV < 0.0093: Consider submitting
- If CV >= 0.0093: Keep exp_007 as final

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.double)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Load data and features
DATA_PATH = '/home/data'

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}')

Spange: (26, 13), DRFP filtered: (24, 122)


In [3]:
# Combined Featurizer
class CombinedFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[["Residence Time", "Temperature"]].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
        
        return torch.tensor(np.hstack([X_kinetic, X_spange, X_drfp]))

print(f'Feature dimension: {CombinedFeaturizer().feats_dim}')

Feature dimension: 140


In [4]:
# Single Hidden Layer [16] MLP
class SingleLayerMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=16, output_dim=3, dropout=0.05):
        super(SingleLayerMLP, self).__init__()
        self.net = nn.Sequential(
            nn.BatchNorm1d(input_dim),
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

class SingleLayerEnsemble(nn.Module):
    def __init__(self, data='single', n_models=5):
        super().__init__()
        self.data_type = data
        self.n_models = n_models
        self.featurizer = CombinedFeaturizer(mixed=(data=='full'))
        self.models = nn.ModuleList()

    def train_model(self, X_train, y_train, epochs=200, batch_size=32, lr=5e-4):
        X_std = self.featurizer.featurize(X_train, flip=False)
        y_vals = torch.tensor(y_train.values)
        
        if self.data_type == 'full':
            X_flip = self.featurizer.featurize(X_train, flip=True)
            X_all = torch.cat([X_std, X_flip], dim=0)
            y_all = torch.cat([y_vals, y_vals], dim=0)
        else:
            X_all = X_std
            y_all = y_vals
            
        input_dim = X_all.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        for i in range(self.n_models):
            torch.manual_seed(42 + i * 13)
            np.random.seed(42 + i * 13)
            
            # Single hidden layer [16]
            model = SingleLayerMLP(input_dim, hidden_dim=16, dropout=0.05).to(device)
            model.train()
            self.models.append(model)
            
            dataset = TensorDataset(X_all, y_all)
            loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            criterion = nn.HuberLoss()
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20)
            
            for epoch in range(epochs):
                epoch_loss = 0.0
                for inputs, targets in loader:
                    inputs, targets = inputs.to(device), targets.to(device)
                    optimizer.zero_grad()
                    outputs = model(inputs)
                    loss = criterion(outputs, targets)
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    optimizer.step()
                    epoch_loss += loss.item() * inputs.size(0)
                scheduler.step(epoch_loss / len(dataset))

    def predict(self, X):
        device = next(self.models[0].parameters()).device
        
        if self.data_type == 'full':
            X_std = self.featurizer.featurize(X, flip=False).to(device)
            X_flip = self.featurizer.featurize(X, flip=True).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += (model(X_std) + model(X_flip)) * 0.5
            avg_pred = pred_sum / self.n_models
        else:
            X_std = self.featurizer.featurize(X).to(device)
            pred_sum = torch.zeros((len(X), 3)).to(device)
            with torch.no_grad():
                for model in self.models:
                    model.eval()
                    pred_sum += model(X_std)
            avg_pred = pred_sum / self.n_models
        return avg_pred.cpu()

print('SingleLayerEnsemble defined: MLP [16], dropout 0.05, 5 models')

SingleLayerEnsemble defined: MLP [16], dropout 0.05, 5 models


In [5]:
# Run CV for single solvent
print("\n--- TASK 0: Single Solvent (24 folds) ---")
X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []
all_actuals = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    model = SingleLayerEnsemble(data='single', n_models=5)
    model.train_model(train_X, train_Y, epochs=200)
    predictions = model.predict(test_X).numpy()
    for row_idx, row in enumerate(predictions):
        all_predictions.append({"task": 0, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]})
    all_actuals.append(test_Y.values)

submission_single = pd.DataFrame(all_predictions)
actuals_single = np.vstack(all_actuals)
preds_single = submission_single[['target_1', 'target_2', 'target_3']].values
mse_single = np.mean((actuals_single - preds_single) ** 2)
print(f'\nSingle Solvent MSE: {mse_single:.6f}')


--- TASK 0: Single Solvent (24 folds) ---


  0%|          | 0/24 [00:00<?, ?it/s]

  4%|▍         | 1/24 [00:47<18:05, 47.18s/it]

  8%|▊         | 2/24 [01:32<16:54, 46.12s/it]

 12%|█▎        | 3/24 [02:15<15:39, 44.72s/it]

...

 92%|█████████▏| 22/24 [16:37<01:31, 45.60s/it]

 96%|█████████▌| 23/24 [17:23<00:45, 45.62s/it]

100%|██████████| 24/24 [18:09<00:00, 45.82s/it]

100%|██████████| 24/24 [18:09<00:00, 45.40s/it]


Single Solvent MSE: 0.009610


In [6]:
# Run CV for full data
print("\n--- TASK 1: Full Data (13 folds) ---")
X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    model = SingleLayerEnsemble(data='full', n_models=5)
    model.train_model(train_X, train_Y, epochs=200)
    predictions = model.predict(test_X).numpy()
    for row_idx, row in enumerate(predictions):
        all_predictions_full.append({"task": 1, "fold": fold_idx, "row": row_idx,
            "target_1": row[0], "target_2": row[1], "target_3": row[2]})
    all_actuals_full.append(test_Y.values)

submission_full = pd.DataFrame(all_predictions_full)
actuals_full = np.vstack(all_actuals_full)
preds_full = submission_full[['target_1', 'target_2', 'target_3']].values
mse_full = np.mean((actuals_full - preds_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f}')


--- TASK 1: Full Data (13 folds) ---


  0%|          | 0/13 [00:00<?, ?it/s]

  8%|▊         | 1/13 [02:37<31:29, 157.46s/it]

 15%|█▌        | 2/13 [05:10<28:24, 154.99s/it]

 23%|██▎       | 3/13 [07:48<26:02, 156.29s/it]

...

 85%|████████▍ | 11/13 [29:04<05:31, 165.63s/it]

 92%|█████████▏| 12/13 [32:13<02:52, 172.73s/it]

100%|██████████| 13/13 [35:21<00:00, 177.12s/it]

100%|██████████| 13/13 [35:21<00:00, 163.17s/it]


Full Data MSE: 0.008969


In [7]:
# Calculate overall MSE
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== FINAL RESULTS ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\n=== SIMPLIFICATION ARC COMPLETE ===')
print(f'  [256,128,64]: CV 0.010430')
print(f'  [64,32]: CV 0.009749')
print(f'  [32,16]: CV 0.009262 (BEST)')
print(f'  [16]: CV {overall_mse:.6f}')
print(f'  Ridge (linear): CV 0.011509')
print(f'\nDecision:')
if overall_mse < 0.009262:
    print(f'  [16] is BETTER than [32,16]! Consider submitting.')
else:
    print(f'  [32,16] remains BEST. exp_007 is the optimal submission.')


=== FINAL RESULTS ===
Single Solvent MSE: 0.009610 (n=656)
Full Data MSE: 0.008969 (n=1227)
Overall MSE: 0.009192

=== SIMPLIFICATION ARC COMPLETE ===
  [256,128,64]: CV 0.010430
  [64,32]: CV 0.009749
  [32,16]: CV 0.009262 (BEST)
  [16]: CV 0.009192
  Ridge (linear): CV 0.011509

Decision:
  [16] is BETTER than [32,16]! Consider submitting.


In [8]:
# Save submission
submission = pd.concat([submission_single, submission_full])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)
print(f'Submission saved: {submission.shape}')
print(submission.head())

Submission saved: (1883, 7)
    index  task  fold  row  target_1  target_2  target_3
id                                                      
0       0     0     0    0  0.011253  0.012687  0.933060
1       1     0     0    1  0.021416  0.022245  0.886226
2       2     0     0    2  0.035762  0.034179  0.813981
3       3     0     0    3  0.051583  0.046147  0.745314
4       4     0     0    4  0.067719  0.057933  0.678407
